In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#trying a very simple model, bad model
cleaned_data = pd.read_csv("data/cleaned.csv")
y = cleaned_data['price']
X = cleaned_data[['distance']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

predict = model.predict(X_test)
print("Train set accuracy: "+str(model.score(X_train, y_train)))
print("Test set accuracy: "+str(model.score(X_test, y_test)))



Train set accuracy: 0.11927643985161185
Test set accuracy: 0.11863362847598147
